# Intro

For generating dataset of shape (n_samples, max_seq_len, feats_per_step) from the original dataframe 

This version is the default

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

import pickle
from datetime import date

### Setup

In [2]:
ds_suffix = "-uk"
max_seq_len = 20
min_seq_len = 5

# Process

In [3]:
df = pd.read_csv("data/data.csv", parse_dates=["Date"])
df.columns = [x.lower() for x in df.columns]
df['age'] = -1
df

,unnamed: 0,index,account_id,amount,balance,date,description,flag,id,type,age
0,1,0,4ecc8ba692c44f0d,-33.00,349.00,2017-04-01 06:00:00,O2 Mobile,Utility Bill,8f45de554a6c80e5,Debit,-1
1,2,0,57f3b8a9c3fb4f9b,-34.25,1012.75,2017-04-01 06:00:00,Water Bill,Utility Bill,074b926e86c30e0b,Debit,-1
2,3,0,aa1b1a1ced0e584a,-33.00,2693.00,2017-04-01 06:00:00,O2 Mobile,Utility Bill,e0925aaf2b7ea5cf,Debit,-1
3,4,0,2c5458f882d7b386,-646.00,-322.00,2017-04-01 06:00:00,Rent,Utility Bill,272bf9b6480d9786,Debit,-1
4,5,0,44ca87a2e9f5276d,-890.00,-473.00,2017-04-01 06:00:00,Rent,Utility Bill,cb5d2710c252ae69,Debit,-1
...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,9,e9e40a02ab7e1537,-433.53,3263.20,2017-05-25 20:00:00,Credit Card Payment,Credit Card,877ce978ce5ef6ae,Debit,-1
99996,99997,12,e472b36546abfbf8,-1645.00,5256.18,2017-05-25 20:00:00,Mortgage,Utility Bill,e9c15f6b653337b6,Debit,-1
99997,99998,9,05e4ab3fdaea4668,-12.95,177.96,2017-05-25 20:00:00,Birch Box,Subscription,ee483e25abfb7a26,Debit,-1
99998,99999,11,664e5cba1356175d,-722.07,-3293.83,2017-05-25 20:00:00,Credit Card Payment,Credit Card,b698d2c34ce56561,Debit,-1


In [4]:
!pwd

/Users/kylenickerson/Dropbox/Mac/Desktop/verafin_proj_big/banksformer_ecml2022/uk/banksformer_v2b_no_conditioning


##### Sort by acct 

In [5]:
df = df.sort_values(by = ["account_id", "date"])

### Tcode

In [6]:
from field_config import cat_code_fields, TCODE_SEP

# create tcode by concating fields in "cat_code_fields"
def set_tcode(df, cat_code_fields):
    tcode = df[cat_code_fields[0]].astype(str)
    for ccf in cat_code_fields[1:]:
        tcode += TCODE_SEP + df[ccf].astype(str)

    df["tcode"] = tcode
    
    
set_tcode(df, cat_code_fields)

DATA_KEY_ORDER is ['tcode_num', 'dow', 'month', 'day', 'dtme', 'td_sc', 'log_amount_sc']
LOSS_TYPES are: day - scce, dtme - scce, dow - scce, month - scce, td_sc - pdf, log_amount_sc - pdf, tcode_num - scce
If this is not correct, edit field_config.py and re-run notebook


##### Date

In [7]:
from datetime import datetime
czech_date_parser = lambda x: datetime.strptime(str(x), "%y%m%d")

df["datetime"] = df["date"] #.apply(czech_date_parser)

iso = df["datetime"].dt.isocalendar()

df["month"] = df["datetime"].dt.month
df["day"] = df["datetime"].dt.day
df["dow"] =  df["datetime"].dt.dayofweek
df["year"] = df["datetime"].dt.year
df

,unnamed: 0,index,account_id,amount,balance,date,description,flag,id,type,age,tcode,datetime,month,day,dow,year
28978,28979,0,0014d0ef29aa9f93,-45.66,228.34,2017-04-24 11:00:00,Energy,Utility Bill,38a21d894127b49d,Debit,-1,Energy__Utility Bill__Debit,2017-04-24 11:00:00,4,24,0,2017
30005,30006,1,0014d0ef29aa9f93,2842.27,3070.61,2017-04-24 17:36:55,Monthly,Income,3941e50e4613a49a,Credit,-1,Monthly__Income__Credit,2017-04-24 17:36:55,4,24,0,2017
32219,32220,2,0014d0ef29aa9f93,-167.25,2903.36,2017-04-25 16:00:00,Credit Card Payment,Credit Card,c09e8105dcfc85b7,Debit,-1,Credit Card Payment__Credit Card__Debit,2017-04-25 16:00:00,4,25,1,2017
35525,35526,3,0014d0ef29aa9f93,-40.58,2862.78,2017-04-27 16:00:00,Water Bill,Utility Bill,81e4ffc8df86920e,Debit,-1,Water Bill__Utility Bill__Debit,2017-04-27 16:00:00,4,27,3,2017
94258,94259,4,0014d0ef29aa9f93,-43.16,2819.62,2017-05-23 17:00:00,Energy,Utility Bill,c0ddd8f8a923d40f,Debit,-1,Energy__Utility Bill__Debit,2017-05-23 17:00:00,5,23,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89756,89757,23,ffd5620bcbf69023,-34.82,2107.16,2017-05-20 15:50:57,Shopping - Household Goods,Card Transaction,908ac96c72f6c3b7,Debit,-1,Shopping - Household Goods__Card Transaction__...,2017-05-20 15:50:57,5,20,5,2017
91384,91385,24,ffd5620bcbf69023,-36.50,2070.66,2017-05-22 08:00:00,Water Bill,Utility Bill,66d16692309a0738,Debit,-1,Water Bill__Utility Bill__Debit,2017-05-22 08:00:00,5,22,0,2017
91448,91449,25,ffd5620bcbf69023,-38.13,2032.53,2017-05-22 09:00:00,Energy,Utility Bill,b71e06b7164af1e3,Debit,-1,Energy__Utility Bill__Debit,2017-05-22 09:00:00,5,22,0,2017
93443,93444,26,ffd5620bcbf69023,-42.59,1989.94,2017-05-22 20:47:15,Bar/Pub,Card Transaction,8a28f7eeff2dbabb,Debit,-1,Bar/Pub__Card Transaction__Debit,2017-05-22 20:47:15,5,22,0,2017


In [8]:
import calendar
# dtme - days till month end
df["dtme"] = df.datetime.apply(lambda dt: calendar.monthrange(dt.year, dt.month)[1] - dt.day)

##### Time delta

In [9]:
df["td"] = df[["account_id", "datetime"]].groupby("account_id").diff()
df["td"] = df["td"].apply(lambda x: x.days)
df["td"].fillna(0.0, inplace=True)
df

,unnamed: 0,index,account_id,amount,balance,date,description,flag,id,type,age,tcode,datetime,month,day,dow,year,dtme,td
28978,28979,0,0014d0ef29aa9f93,-45.66,228.34,2017-04-24 11:00:00,Energy,Utility Bill,38a21d894127b49d,Debit,-1,Energy__Utility Bill__Debit,2017-04-24 11:00:00,4,24,0,2017,6,0.0
30005,30006,1,0014d0ef29aa9f93,2842.27,3070.61,2017-04-24 17:36:55,Monthly,Income,3941e50e4613a49a,Credit,-1,Monthly__Income__Credit,2017-04-24 17:36:55,4,24,0,2017,6,0.0
32219,32220,2,0014d0ef29aa9f93,-167.25,2903.36,2017-04-25 16:00:00,Credit Card Payment,Credit Card,c09e8105dcfc85b7,Debit,-1,Credit Card Payment__Credit Card__Debit,2017-04-25 16:00:00,4,25,1,2017,5,0.0
35525,35526,3,0014d0ef29aa9f93,-40.58,2862.78,2017-04-27 16:00:00,Water Bill,Utility Bill,81e4ffc8df86920e,Debit,-1,Water Bill__Utility Bill__Debit,2017-04-27 16:00:00,4,27,3,2017,3,2.0
94258,94259,4,0014d0ef29aa9f93,-43.16,2819.62,2017-05-23 17:00:00,Energy,Utility Bill,c0ddd8f8a923d40f,Debit,-1,Energy__Utility Bill__Debit,2017-05-23 17:00:00,5,23,1,2017,8,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89756,89757,23,ffd5620bcbf69023,-34.82,2107.16,2017-05-20 15:50:57,Shopping - Household Goods,Card Transaction,908ac96c72f6c3b7,Debit,-1,Shopping - Household Goods__Card Transaction__...,2017-05-20 15:50:57,5,20,5,2017,11,0.0
91384,91385,24,ffd5620bcbf69023,-36.50,2070.66,2017-05-22 08:00:00,Water Bill,Utility Bill,66d16692309a0738,Debit,-1,Water Bill__Utility Bill__Debit,2017-05-22 08:00:00,5,22,0,2017,9,1.0
91448,91449,25,ffd5620bcbf69023,-38.13,2032.53,2017-05-22 09:00:00,Energy,Utility Bill,b71e06b7164af1e3,Debit,-1,Energy__Utility Bill__Debit,2017-05-22 09:00:00,5,22,0,2017,9,0.0
93443,93444,26,ffd5620bcbf69023,-42.59,1989.94,2017-05-22 20:47:15,Bar/Pub,Card Transaction,8a28f7eeff2dbabb,Debit,-1,Bar/Pub__Card Transaction__Debit,2017-05-22 20:47:15,5,22,0,2017,9,0.0


# Write

In [10]:
folders = ['generated_data',
 'generated_data/parts',
 'stored_data',
 'generation_results',
 'data',
 'my_lib']


for f in folders:
    if not os.path.exists(f):
        os.mkdir(f)

In [11]:
df.to_csv(f"stored_data/final_df-{ds_suffix}.csv", index=False)